In [ ]:
import pandas as pd
from itertools import product
import openpyxl
from openpyxl.styles import PatternFill
import random
from collections import defaultdict

In [ ]:
file_name = 'punnett_square.xlsx'

In [66]:
# input the number of crosses
num_crosses = int(input("Please enter the number of crosses: "))

# generate the genes assuming two alleles per gene (e.g., Aa, Bb)
genes = [chr(65+i)+chr(97+i) for i in range(num_crosses)]

# generate all possible combinations of alleles
gametes = [''.join(gamete) for gamete in product(*genes)]

# initialize an empty DataFrame
df = pd.DataFrame(index=gametes, columns=gametes)

def sort_characters(text):
    # sort the characters alphabetically, then by uppercase-lowercase within each alphabet group
    sorted_characters = sorted(text, key=lambda char: (char.lower(), not char.isupper()))

    # join the sorted characters back into a string
    return ''.join(sorted_characters)

# fill in the DataFrame with combinations of gametes
genotypes = []
for i in gametes:
    for j in gametes:
        # combine and sort each pair of alleles, then join them back into strings
        gamete = sort_characters(''.join(sorted(i[k]+j[k] for k in range(len(i)))))
        genotypes.append(gamete)
        df.loc[i,j] = gamete

# write to Excel file
df.to_excel(file_name)

# done!
print(f'done with {len(genotypes)}(2^{2*num_crosses}) genotypes!')


Please enter the number of crosses: 2
done with 16(2^4) genotypes!


In [67]:
def generate_random_color():
  r = lambda: random.randint(128,255)
  return '{:02X}{:02X}{:02X}'.format(r(),r(),r())

# load workbook and select active sheet
wb = openpyxl.load_workbook(file_name)
ws = wb.active

# create a dictionary to store cell values and their corresponding colors
cell_dict = {}

for row in ws.iter_rows(min_row=2):
    for cell in row:
        if cell.column == 1:
            continue

        if cell.value not in cell_dict:
            # generate a new color for this unique value
            color_code = generate_random_color()
            fill_color = PatternFill(start_color=color_code,
                                    end_color=color_code,
                                    fill_type="solid")
            cell_dict[cell.value] = fill_color

        # fill the cell with its corresponding color
        cell.fill = cell_dict[cell.value]

# write to new Excel file
wb.save('colored_' + file_name)

# done!
print(f'done with beautiful {len(cell_dict)} colors!')


done with beautiful 9 colors!


In [68]:
# initialize the phenotype counter
phenotypes = defaultdict(int)

# count phenotypes
for genotype in genotypes:
    phenotype = ''

    for i in range(0, len(genotype), 2):
        if genotype[i].isupper():
            trait = genotypes[0][2*(i//2+1)-1]
        else:
            trait = (genotypes[0][2*(i//2+1)-1]).lower()

        phenotype += trait

    phenotypes[phenotype] += 1

print('============================')
print('    < phenotypic ratio >    ')
print('* uppercase: dominant  trait')
print('* lowercase: recessive trait')
print('============================')

# sort and print results in descending order
sorted_result = sorted(phenotypes.items(), key=lambda x: x[1], reverse=True)
for k, v in sorted_result:
     print(f"{k}: {v}")


    < phenotypic ratio >    
* uppercase: dominant  trait
* lowercase: recessive trait
AB: 9
Ab: 3
aB: 3
ab: 1
